# Simple linear regression

In this notebook, we take a closer look at simple linear regression

First we import the standard packages

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats

Then we import some example data. In this case Google Analytics webdata about daily users and daily number of purchases in the webshop

In [ ]:
webdata = pd.read_excel("GA users and convertions.xlsx")

In [ ]:
webdata

## Visualizing simple linear regression

To visualize the best fitted linear regression line in a scatterplot with Seaborn, we use the function `regplot`.

In [ ]:
sns.regplot(data = webdata, x = "Users", y = "PurchaseCompleted")
plt.title("Linear regression plot for users and purchases completed")
plt.show()

The shaded area along the regression line is the *confidence interval*, which we will not talk more about for now, but essentially it visualizes the uncertainly associated the regression line. We can remove the confidence interval from our plot if we want:

In [ ]:
sns.regplot(data = webdata, x = "Users", y = "PurchaseCompleted", ci=None)
plt.title("Linear regression plot for users and purchases completed")
plt.savefig('corrplot_w_regline.png')
plt.show()

## Fitting a simple linear regression model using OLS

We can fit a simple regression line to our data such that the sum of squared errors (residuals) are minimized. We call this approach *"Ordinary Least Squares"* (OLS)

We will first use the [statsmodels](https://www.statsmodels.org/stable/index.html) package to fit a OLS model to our data. See also https://www.statsmodels.org/stable/regression.html

First we import the relevant package

In [ ]:
import statsmodels.api as sm

We define our *Users* column to be our X data.

In [ ]:
X = webdata["Users"]

In [ ]:
X

Using statsmodels, we have to add the intercept (in the form of a column of 1s)

In [ ]:
X = sm.add_constant(X)

In [ ]:
X

We then set the *PurchaseCompleted* column to our y.

In [ ]:
y = webdata["PurchaseCompleted"]

We can now fit a simple linear regression model using OLS

In [ ]:
linreg_model = sm.OLS(y, X).fit()

We can get a lot of information about our model from the summary method.

In [ ]:
linreg_model.summary()

**Interpretation of the model**

Here we see general information about the model at the top, such as R-squared (0.378) and  Adjusted R-squared (0.372). That is, our model explain 37.8% of the variance in purchases completed by the variance in users.

This is followed by information about each of the coefficients (including the intercept named "const"). The *coef* column is the value for the coefficient found by OLS. The *P>|t|* column give us the p-value associated with that column - if it is less than our significance level, the cofficient is statistically different from 0. This is the most interesting information about the model for now.

We see that for each additional user, we can expect 0.0297 purchases completed, that is for 100 additional users we will expect to see almost 3 purchases completed (100*0.0297). With a p-value of basically 0, we can also conclude that this relationship is indeed statistically significant. The intercept of -2.3670 is not quite significant as the p-values is 0.074, which is above 0.05.

### Retrieving coefficients and plotting fitted regression line

We can also get the parameters or coefficients from the fitted model the following way:

In [ ]:
linreg_model.params

We can then use that to manually calculate the prediction for each point in webdata:

In [ ]:
pred_y = linreg_model.params["const"] + linreg_model.params["Users"]*webdata["Users"]
pred_y

We can now manually add the OLS regression line to our scatter plot

In [ ]:
sns.scatterplot(data = webdata, x = "Users", y = "PurchaseCompleted")
plt.plot(webdata["Users"], pred_y, color = "orange")
plt.show()

Which looks the same as the regression plot we got from Seaborns regplot:

In [ ]:
sns.regplot(data = webdata, x = "Users", y = "PurchaseCompleted", ci=None)
plt.show()

We could also have gotten the prediction on our data directly from the model object using the predict method.

In [ ]:
linreg_model.predict()

We can verify that they are the same as our manually calculated predictions.

In [ ]:
pred_test_df = pd.DataFrame({"PredictMethod": linreg_model.predict(), "ManuallyCalculated" : pred_y})
pred_test_df["Difference"] = pred_test_df["PredictMethod"] - pred_test_df["ManuallyCalculated"]
pred_test_df

In [ ]:
sum(pred_test_df["Difference"])

Apparently there are some very small rounding errors somewhere, but they are not significant.

### Model performance measures

We can also get the R-squared directly from the fitted model object.

In [ ]:
linreg_model.rsquared

In [ ]:
linreg_model.rsquared_adj

We can get the residuals of the model also

In [ ]:
linreg_model.resid

Using these, we can calculate the *MAE* (Mean Absolute Error).

In [ ]:
np.mean(np.abs(linreg_model.resid))

and the *MSE* (Mean Squared Error).

In [ ]:
np.mean(linreg_model.resid**2)

and the *RMSE* (Root Mean Squared Error).

In [ ]:
np.sqrt(np.mean(linreg_model.resid**2))

## Fitting simple linear regression using scikit-learn

We can also use the machine learning package [scikit-learn](https://scikit-learn.org/stable/) to fit a linear regression model (also using OLS). First we import the linear_model submodule.

In [ ]:
from sklearn import linear_model

Then we define a linear regression mode.

In [ ]:
linreg_model_sk = linear_model.LinearRegression()

For scikit-learn we do not need to have an intercept column explicitly in our X dataset. However, we still need to have it as a pandas dataframe. Thus we define a new X first.

In [ ]:
X_no_int = webdata[["Users"]]

We can now fit the model.

In [ ]:
linreg_model_sk.fit(X_no_int, y)

Getting the coefficient in scikit-learn will give us all the coefficient except the coefficient for the intercept. That we have to get seperately.

In [ ]:
linreg_model_sk.coef_

In [ ]:
linreg_model_sk.intercept_

We see that we get the same value as for statsmodels OLS.

To get the prediction of the model, we can use the `.predict` method. Note that we have to give the method some X data to predict on. This is useful if one have trained the model on some training data and want to predict for some test data.

In [ ]:
linreg_model_sk.predict(X_no_int)

We can also get the R-squared, but here we need to provide data. This is useful if one have trained a model on some training data and wants to calculate R-squared on some test data.

In [ ]:
linreg_model_sk.score(X_no_int, y)

Scikit-learn also comes with a metrics submodule with all sorts of evaluation metrics (See https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics)

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error

All of these take the true and the predicted values as arrays. Thus, we first define a variable for the predicted.

In [ ]:
y_pred_sk = linreg_model_sk.predict(X_no_int)

In [ ]:
r2_score(y, y_pred_sk)

In [ ]:
mean_absolute_error(y, y_pred_sk)

In [ ]:
mean_squared_error(y, y_pred_sk)

In [ ]:
root_mean_squared_error(y, y_pred_sk)

If you use linear regression as a machine learning model for prediction, don't forget to do a train-test split of your data. (More about this next time!)